In [56]:
from typing import List, get_origin
from guardrails.validators import LowerCase, OneLine, TwoWords

from lxml.builder import E
from lxml.etree import Element, tostring

In [55]:
from collections import defaultdict
from datetime import date, time
import logging
from typing import TYPE_CHECKING, Any, Dict, Optional, Type, Union, get_args, get_origin

from lxml.etree import Element
from lxml.builder import E

from griffe.dataclasses import Docstring
from griffe.docstrings.parsers import Parser, parse
import pydantic

from pydantic import BaseModel, Field, create_model
from pydantic.fields import ModelField

from guardrails.utils.pydantic_utils import add_validator

In [2]:
from typing import Dict, get_args, get_type_hints


class FeeDetailsReask(BaseModel):
    index: int = Field(gd_validators="1-indexed")
    name: str = Field(gd_validators=[LowerCase(on_fail="reask"), TwoWords()])
    explanation: str = Field(gd_validators=OneLine(on_fail="reask"))
    value: float = Field(gd_validators="percentage")


class ContractDetailsReask(BaseModel):
    # fees: List[FeeDetailsReask] = Field(
    #     description="What fees and charges are associated with my account?"
    # )
    interest_rates: List[str] = Field(
        description="What are the interest rates offered by the bank on savings and checking accounts, loans, and credit products?"
    )

NameError: name 'GuardModel' is not defined

In [4]:
# from datetime import date, time
# from typing import Type, Union

# from pydantic import BaseModel, HttpUrl
# import pydantic


# def is_pydantic_base_model(type_annotation: Type) -> bool:
#     """Check if a type_annotation is a Pydantic BaseModel."""
#     try:
#         if issubclass(type_annotation, BaseModel):
#             return True
#     except TypeError:
#         False
#     return False


# def get_model_field_annotation(type_annotation: Union[Type, pydantic.fields.ModelField]) -> Type:
#     """Get the type annotation from a ModelField."""

#     if isinstance(type_annotation, pydantic.fields.ModelField):
#         type_annotation = type_annotation.annotation

#     return type_annotation


# def is_list(type_annotation: Type) -> bool:
#     """Check if a type_annotation is a list."""

#     type_annotation = get_model_field_annotation(type_annotation)

#     if is_pydantic_base_model(type_annotation):
#         return False
#     if get_origin(type_annotation) == list:
#         return True
#     elif type_annotation == list:
#         return True
#     return False


# def is_dict(type_annotation: Type) -> bool:
#     """Check if a type_annotation is a dict."""

#     type_annotation = get_model_field_annotation(type_annotation)

#     if is_pydantic_base_model(type_annotation):
#         return True
#     if get_origin(type_annotation) == dict:
#         return True
#     elif type_annotation == dict:
#         return True
#     return False


# def map_field_info_to_field_type(field_info: Union[pydantic.fields.ModelField, Type]):
#     # Get the type annotation from the field_info
#     if isinstance(field_info, pydantic.fields.ModelField):
#         type_annotation = field_info.annotation
#     elif type(field_info) == type:
#         type_annotation = field_info
#     elif get_origin(field_info) is not None:
#         type_annotation = get_origin(field_info)
#     else:
#         raise ValueError(f"Unsupported type: {field_info}")

#     # Map the type annotation to the corresponding field type
#     if is_list(type_annotation):
#         return "list"
#     elif is_dict(type_annotation):
#         return "object"
#     elif type_annotation == bool:
#         return "bool"
#     elif type_annotation == date:
#         return "date"
#     elif type_annotation == float:
#         return "float"
#     elif type_annotation == int:
#         return "integer"
#     elif type_annotation == str:
#         return "string"
#     elif type_annotation == time:
#         return "time"
#     elif type_annotation == HttpUrl:
#         return "url"
#     else:
#         raise ValueError(f"Unsupported type: {type_annotation}")

In [5]:
# from typing import Optional
# from lxml.etree import Element, tostring


# def create_xml_element_for_field(
#     field_info: Union[pydantic.fields.ModelField, Type, type],
#     field_name: Optional[str] = None,
# ) -> Element:
#     """Create an XML element corresponding to a field.

#     Args:
#         field_info: Field's type. This could be a Pydantic ModelField or a type.
#         field_name: Field's name. For some fields (e.g. list), this is not required.

#     Returns:
#         The XML element corresponding to the field.
#     """
#     field_type = map_field_info_to_field_type(field_info)

#     element = E(field_type)
#     if field_name:
#         element.set("name", field_name)

#     # Extract validators from the Pydantic field.
#     # Add format and on-fail attributes
#     if (
#         isinstance(field_info, pydantic.fields.ModelField)
#         and hasattr(field_info.field_info, "gd_validators")
#         and field_info.field_info.gd_validators is not None
#     ):
#         format_prompt = []
#         on_fails = {}
#         for validator in field_info.field_info.gd_validators:
#             validator_prompt = validator
#             if not isinstance(validator, str):
#                 # `validator`` is of type gd.Validator, use the to_xml_attrib method
#                 validator_prompt = validator.to_xml_attrib()
#                 # Set the on-fail attribute based on the on_fail value
#                 on_fail = validator.on_fail.__name__ if validator.on_fail else "noop"
#                 on_fails[validator.rail_alias] = on_fail
#             format_prompt.append(validator_prompt)

#         if len(format_prompt) > 0:
#             format_prompt = "; ".join(format_prompt)
#             element.set("format", format_prompt)
#             for rail_alias, on_fail in on_fails.items():
#                 element.set("on-fail-" + rail_alias, on_fail)

#     # Add description attribute
#     if (
#         isinstance(field_info, pydantic.fields.ModelField)
#         and field_info.field_info.description is not None
#     ):
#         element.set("description", field_info.field_info.description)

#     # Handle nested types
#     if field_type in ["list", "object"]:
#         type_annotation = get_model_field_annotation(field_info)
#         # Check for all the ways a list can be defined
#         if is_list(type_annotation):
#             inner_type = get_args(type_annotation)
#             if len(inner_type) > 0:
#                 inner_type = inner_type[0]
#                 if is_pydantic_base_model(inner_type):
#                     object_element = E("object")
#                     for inner_field_name, inner_field_info in inner_type.__fields__.items():
#                         inner_element = create_xml_element_for_field(
#                             inner_field_info, inner_field_name
#                         )
#                         object_element.append(inner_element)
#                     element.append(object_element)
#                 else:
#                     inner_element = create_xml_element_for_field(inner_type)
#                     element.append(inner_element)
#         elif is_dict(type_annotation):
#             if is_pydantic_base_model(type_annotation):
#                 for inner_field_name, inner_field_info in type_annotation.__fields__.items():
#                     inner_element = create_xml_element_for_field(
#                         inner_field_info, inner_field_name
#                     )
#                     element.append(inner_element)
#             else:
#                 dict_args = get_args(type_annotation)
#                 if len(dict_args) == 2:
#                     key_type, val_type = get_args(type_annotation)
#                     assert key_type == str, "Only string keys are supported for dicts"
#                     inner_element = create_xml_element_for_field(val_type)
#                     element.append(inner_element)
#         else:
#             raise ValueError(f"Unsupported type: {type_annotation}")

#     return element

In [6]:
# outermost_element = E("output")

# for field_name, field_info in ContractDetailsReask.__fields__.items():
#     element = create_xml_element_for_field(field_info, field_name)
#     outermost_element.append(element)

In [28]:
# print(tostring(
#     outermost_element,
#     encoding="unicode",
#     method="xml",
#     pretty_print=True,
# ))

<output>
  <list name="fees" description="What fees and charges are associated with my account?">
    <object>
      <integer name="index" format="1-indexed"/>
      <string name="name" format="two-words; lower-case" on-fail-two-words="noop" on-fail-lower-case="reask"/>
      <string name="explanation" format="one-line" on-fail-one-line="reask"/>
      <float name="value" format="percentage"/>
    </object>
  </list>
  <list name="interest_rates" description="What are the interest rates offered by the bank on savings and checking accounts, loans, and credit products?">
    <string/>
  </list>
  <object name="dummy_other_thing" description="What fees and charges are associated with my account?">
    <integer name="index" format="1-indexed"/>
    <string name="name" format="two-words; lower-case" on-fail-two-words="noop" on-fail-lower-case="reask"/>
    <string name="explanation" format="one-line" on-fail-one-line="reask"/>
    <float name="value" format="percentage"/>
  </object>
  <obj

In [42]:
def is_pydantic_base_model(type_annotation: Any) -> bool:
    """Check if a type_annotation is a Pydantic BaseModel."""
    try:
        if issubclass(type_annotation, BaseModel):
            return True
    except TypeError:
        False
    return False


def is_list(type_annotation: Any) -> bool:
    """Check if a type_annotation is a list."""

    type_annotation = prepare_type_annotation(type_annotation)

    if is_pydantic_base_model(type_annotation):
        return False
    if get_origin(type_annotation) == list:
        return True
    elif type_annotation == list:
        return True
    return False


def is_dict(type_annotation: Any) -> bool:
    """Check if a type_annotation is a dict."""

    type_annotation = prepare_type_annotation(type_annotation)

    if is_pydantic_base_model(type_annotation):
        return True
    if get_origin(type_annotation) == dict:
        return True
    elif type_annotation == dict:
        return True
    return False


def prepare_type_annotation(type_annotation: Any) -> Type:
    """Get the raw type annotation that can be used for downstream processing.

    This function does the following:
        1. If the type_annotation is a Pydantic field, get the annotation
        2. If the type_annotation is a Union, get the first non-None type

    Args:
        type_annotation (Any): The type annotation to prepare

    Returns:
        Type: The prepared type annotation
    """

    if isinstance(type_annotation, ModelField):
        type_annotation = type_annotation.annotation

    # Strip a Union type annotation to the first non-None type
    if get_origin(type_annotation) == Union:
        type_annotation = [t for t in get_args(type_annotation) if t != type(None)]
        assert (
            len(type_annotation) == 1
        ), "Union type must have exactly one non-None type"
        type_annotation = type_annotation[0]

    return type_annotation


def type_annotation_to_string(type_annotation: Any) -> str:
    """Map a type_annotation to the name of the corresponding field type.

    This function checks if the type_annotation is a list, dict, or a primitive type,
    and returns the corresponding type name, e.g. "list", "object", "bool", "date", etc.
    """

    # Get the type annotation from the type_annotation
    type_annotation = prepare_type_annotation(type_annotation)

    # Map the type annotation to the corresponding field type
    if is_list(type_annotation):
        return "list"
    elif is_dict(type_annotation):
        return "object"
    elif type_annotation == bool:
        return "bool"
    elif type_annotation == date:
        return "date"
    elif type_annotation == float:
        return "float"
    elif type_annotation == int:
        return "integer"
    elif type_annotation == str:
        return "string"
    elif type_annotation == time:
        return "time"
    elif type_annotation == HttpUrl:
        return "url"
    else:
        raise ValueError(f"Unsupported type: {type_annotation}")


def add_validators_to_xml_element(field_info: ModelField, element: Element) -> Element:
    """Extract validators from a pydantic ModelField and add to XML element

    Args:
        field_info: The field info to extract validators from
        element: The XML element to add the validators to

    Returns:
        The XML element with the validators added
    """

    if (
        isinstance(field_info, ModelField)
        and hasattr(field_info.field_info, "gd_validators")
        and field_info.field_info.gd_validators is not None
    ):
        format_prompt = []
        on_fails = {}
        for validator in field_info.field_info.gd_validators:
            validator_prompt = validator
            if not isinstance(validator, str):
                # `validator`` is of type gd.Validator, use the to_xml_attrib method
                validator_prompt = validator.to_xml_attrib()
                # Set the on-fail attribute based on the on_fail value
                on_fail = validator.on_fail.__name__ if validator.on_fail else "noop"
                on_fails[validator.rail_alias] = on_fail
            format_prompt.append(validator_prompt)

        if len(format_prompt) > 0:
            format_prompt = "; ".join(format_prompt)
            element.set("format", format_prompt)
            for rail_alias, on_fail in on_fails.items():
                element.set("on-fail-" + rail_alias, on_fail)

    return element


def create_xml_element_for_field(
    field: Union[ModelField, Type, type],
    field_name: Optional[str] = None,
) -> Element:
    """Create an XML element corresponding to a field.

    Args:
        field_info: Field's type. This could be a Pydantic ModelField or a type.
        field_name: Field's name. For some fields (e.g. list), this is not required.

    Returns:
        The XML element corresponding to the field.
    """

    # Create the element based on the field type
    field_type = type_annotation_to_string(field)
    element = E(field_type)

    # Add name attribute
    if field_name:
        element.set("name", field_name)

    # Add validators
    element = add_validators_to_xml_element(field, element)

    # Add description attribute
    if isinstance(field, ModelField):
        if field.field_info.description is not None:
            element.set("description", field.field_info.description)

    # Create XML elements for the field's children
    if field_type in ["list", "object"]:
        type_annotation = prepare_type_annotation(field)

        if is_list(type_annotation):
            inner_type = get_args(type_annotation)
            if len(inner_type) == 0:
                # If the list is empty, we cannot infer the type of the elements
                pass

            inner_type = inner_type[0]
            if is_pydantic_base_model(inner_type):
                object_element = create_xml_element_for_base_model(inner_type)
                element.append(object_element)
            else:
                inner_element = create_xml_element_for_field(inner_type)
                element.append(inner_element)

        elif is_dict(type_annotation):
            if is_pydantic_base_model(type_annotation):
                element = create_xml_element_for_base_model(type_annotation, element)
            else:
                dict_args = get_args(type_annotation)
                if len(dict_args) == 0:
                    # If the dict is empty, we cannot infer the type of the elements
                    pass

                key_type, val_type = dict_args
                assert key_type == str, "Only string keys are supported for dicts"
                inner_element = create_xml_element_for_field(val_type)
                element.append(inner_element)
        else:
            raise ValueError(f"Unsupported type: {type_annotation}")

    return element


def create_xml_element_for_base_model(
    model: BaseModel, element: Optional[Element] = None
) -> Element:
    """Create an XML element for a Pydantic BaseModel.

    This function does the following:
        1. Iterates through fields of the model and creates XML elements for each field
        2. If a field is a Pydantic BaseModel, it creates a nested XML element
        3. If the BaseModel contains a field with a `when` attribute, it creates
           `Choice` and `Case` elements for the field.

    Args:
        model: The Pydantic BaseModel to create an XML element for
        element: The XML element to add the fields to. If None, a new XML element

    Returns:
        The XML element with the fields added
    """

    if element is None:
        element = E("object")

    # Identify fields with `when` attribute
    choice_elements = defaultdict(list)
    case_elements = set()
    for field_name, field in model.__fields__.items():
        if hasattr(field.field_info, "when") and field.field_info.when:
            choice_elements[field.field_info.when].append((field_name, field))
            case_elements.add(field_name)

    # Add fields to the XML element, except for fields with `when` attribute
    for field_name, field in model.__fields__.items():
        if field_name in choice_elements or field_name in case_elements:
            continue
        field_element = create_xml_element_for_field(field, field_name)
        element.append(field_element)

    # Add `Choice` and `Case` elements for fields with `when` attribute
    for when, discriminator_fields in choice_elements.items():
        choice_element = E("choice", name=when)
        # TODO(shreya): DONT MERGE WTHOUT SOLVING THIS: How do you set this via SDK?
        choice_element.set("on-fail-choice", "exception")

        for field_name, field in discriminator_fields:
            case_element = E("case", name=field_name)
            field_element = create_xml_element_for_field(field, field_name)
            case_element.append(field_element)
            choice_element.append(case_element)

        element.append(choice_element)

    return element

In [6]:
from guardrails.validators import ValidChoices


class FlightDetails(BaseModel):
    flight_direction: str = Field(
        validators=ValidChoices(
            choices=["north", "south", "east", "west"], on_fail="exception"
        )
    )
    flight_speed: int = Field(
        validators=ValidChoices(choices=[1, 2, 3, 4], on_fail="exception")
    )


class Action(BaseModel):
    action: str = Field(validators=ValidChoices(choices=["fight", "flight"]))
    fight: Optional[str] = Field(
        validators=ValidChoices(choices=["punch", "kick"], on_fail="exception"),
        when="action",
    )
    flight: Optional[FlightDetails] = Field(when="action")

In [29]:
output = base_model_iterator(Action)

In [43]:
output = create_xml_element_for_base_model(ContractDetailsReask)

In [41]:
print(tostring(
    output,
    encoding="unicode",
    method="xml",
    pretty_print=True,
))

<object>
  <list name="interest_rates" description="What are the interest rates offered by the bank on savings and checking accounts, loans, and credit products?">
    <string/>
  </list>
</object>



In [56]:
from decimal import Decimal

from pydantic import (
    BaseModel,
    NegativeFloat,
    NegativeInt,
    PositiveFloat,
    PositiveInt,
    NonNegativeFloat,
    NonNegativeInt,
    NonPositiveFloat,
    NonPositiveInt,
    conbytes,
    condecimal,
    confloat,
    conint,
    conlist,
    conset,
    constr,
    Field,
)


class Model(BaseModel):
    upper_bytes: conbytes(to_upper=True)
    lower_bytes: conbytes(to_lower=True)
    short_bytes: conbytes(min_length=2, max_length=10)
    strip_bytes: conbytes(strip_whitespace=True)

    upper_str: constr(to_upper=True)
    lower_str: constr(to_lower=True)
    short_str: constr(min_length=2, max_length=10)
    regex_str: constr(regex=r'^apple (pie|tart|sandwich)$')
    strip_str: constr(strip_whitespace=True)

    big_int: conint(gt=1000, lt=1024)
    mod_int: conint(multiple_of=5)
    pos_int: PositiveInt
    neg_int: NegativeInt
    non_neg_int: NonNegativeInt
    non_pos_int: NonPositiveInt

    big_float: confloat(gt=1000, lt=1024)
    unit_interval: confloat(ge=0, le=1)
    mod_float: confloat(multiple_of=0.5)
    pos_float: PositiveFloat
    neg_float: NegativeFloat
    non_neg_float: NonNegativeFloat
    non_pos_float: NonPositiveFloat

    short_list: conlist(int, min_items=1, max_items=4)
    short_set: conset(int, min_items=1, max_items=4)

    decimal_positive: condecimal(gt=0)
    decimal_negative: condecimal(lt=0)
    decimal_max_digits_and_places: condecimal(max_digits=2, decimal_places=2)
    mod_decimal: condecimal(multiple_of=Decimal('0.25'))

    bigger_int: int = Field(..., gt=10000, random_other_thing=123)

In [57]:
dir(Model.__fields__["mod_int"].annotation)

['__abs__',
 '__add__',
 '__and__',
 '__bool__',
 '__ceil__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floor__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__get_validators__',
 '__getattribute__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__le__',
 '__lshift__',
 '__lt__',
 '__mod__',
 '__modify_schema__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdivmod__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rlshift__',
 '__rmod__',
 '__rmul__',
 '__ror__',
 '__round__',
 '__rpow__',
 '__rrshift__',
 '__rshift__',
 '__rsub__',
 '__rtruediv__',
 '__rxor__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__trunc__',
 '__xor__',
 'as_integer_ratio',
 'bit_length',
 'conjugate',
 'denominator'

In [63]:
Model.__fields__["bigger_int"].field_info.extra['random_other_thing']

123

In [61]:
from pydantic import validator


class RandomBaseModel(BaseModel):
    random_field: str = Field(..., validators=[ValidChoices(choices=["a", "b", "c"])])
    random_field2: str = Field(..., description="This is a random field")

    @validator("random_field")
    def random_field_validator(cls, v):
        # Check if the field is a string that is not "a"
        if isinstance(v, str) and v == "a":
            raise ValueError("Field must not be 'a'")

    @validator("random_field2")
    def random_field2_validator(cls, v):
        # Check if field is an integer that is a multiple of 5
        if isinstance(v, int) and v % 5 == 0:
            raise ValueError("Field must be an integer that is not a multiple of 5")
        return v
    
    # Create a validator for both random_field and random_field2 to check if they are the same
    @validator("random_field", "random_field2")
    def random_field_and_random_field2_validator(cls, v, values):
        # Check if both fields are the same
        if values["random_field"] != values["random_field2"]:
            raise ValueError("Fields must be the same")
        return v

In [63]:
errors = []
for validator in RandomBaseModel.__fields__["random_field"].post_validators:
    print(validator)
    
    # input_data = validator(RandomBaseModel, {"random_field": "a", "random_field2": 10})

<cyfunction RandomBaseModel.random_field_validator at 0x12bc986c0>
<cyfunction RandomBaseModel.random_field_and_random_field2_validator at 0x12bc98860>


In [33]:
import inspect

In [94]:
def extract_pydantic_validators(model: BaseModel):
    validators = defaultdict(list)

    for field in model.__fields__.values():
        if field.pre_validators:
            for val in field.pre_validators:
                fn = getattr(model, val.func_name)
                validators[fn].append(field.name)

    for field in model.__fields__.values():
        if field.post_validators:
            for val in field.post_validators:
                fn = getattr(model, val.func_name)
                validators[fn].append(field.name)

    # TODO(shreya): Before merging handle root validators

    return dict(validators)

In [96]:
for fn, args in extract_pydantic_validators(RandomBaseModel).items():
    print(fn.func_name)

AttributeError: 'function' object has no attribute 'func_name'

In [101]:
fn.__func__.__name__

'random_field_validator'

In [40]:
pydantic_validator = RandomBaseModel.__fields__["random_field"].post_validators[0]

# Call the validator with the class and the value
input_data = pydantic_validator(RandomBaseModel, {"random_field": "a", "random_field2": 10}, "random_field", "a", None)

In [76]:
pydantic_validator.func_name

'random_field_validator'

In [68]:
getattr(RandomBaseModel, "random_field2_validator")(10)

ValueError: Field must be an integer that is not a multiple of 5

In [60]:
dir(getattr(RandomBaseModel, "random_field_validator"))

['__call__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__func__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__self__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [53]:
RandomBaseModel.random_field_validator("a")

ValueError: Field must not be 'a'

In [47]:
# print(inspect.getsource(pydantic_validator))

# Call the validator with the class and the value
pydantic_validator(RandomBaseModel, {"random_field": "a", "random_field2": 10}, "dsf;jhsF", "a", None)

In [48]:
input_data

In [35]:
print(inspect.signature(RandomBaseModel.__fields__["random_field"].post_validators[0]))

(cls, v)


In [17]:
RandomBaseModel(random_field="b", random_field2=9)

RandomBaseModel(random_field=None, random_field2='9')

In [14]:
RandomBaseModel.__fields__["random_field"].post_validators

[<cyfunction RandomBaseModel.random_field_validator at 0x12bc981e0>]

In [70]:
RandomBaseModel.__fields__["random_field2"].field_info.extra

{}

In [71]:
RandomBaseModel.__fields__["random_field"].field_info.extra['validators']

{'validators': [<guardrails.validators.ValidChoices at 0x136b6ea60>]}

In [7]:
x = ValidChoices(choices=["box_office", "streaming"])

In [8]:
x(None, "box_office", None)

# Extracting Pydantic validators

In [9]:
# class BoxOfficeRevenue(BaseModel):
#     gross: float
#     opening_weekend: float

#     # Field-level validation using Pydantic (not Guardrails)
#     @validator("gross")
#     def validate_gross(cls, gross):
#         if gross <= 0:
#             raise ValueError("Gross revenue must be a positive value")
#         return gross

# class StreamingRevenue(BaseModel):
#     subscriptions: int
#     subscription_fee: float

# class Details(BaseModel):
#     release_date: date
#     duration: time
#     budget: float
#     is_sequel: bool = Field(default=False)
#     website: str
#     contact_email: str
#     revenue_type: str
#     box_office: Optional[BoxOfficeRevenue] = Field(when="revenue_type")
#     streaming: Optional[StreamingRevenue] = Field(when="revenue_type")

#     # Register guardrails validators
#     _revenue_type_validator = add_validator(
#         "revenue_type", fn=ValidChoices(choices=["box_office", "streaming"])
#     )
#     _website_validator = add_validator(
#         "website", fn=ValidLength(min=9, max=100, on_fail="reask")
#     )

#     # Root-level validation using Pydantic (Not in Guardrails)
#     @root_validator
#     def validate_budget_and_gross(cls, values):
#         budget = values.get("budget")
#         revenue_type = values.get("revenue_type")
#         box_office_revenue = values.get("box_office")
#         if revenue_type == "box_office" and box_office_revenue:
#             gross = box_office_revenue.gross
#             if budget >= gross:
#                 raise ValueError("Budget must be less than gross revenue")
#         return values

# class Movie(BaseModel):
#     rank: int
#     title: str
#     details: Details

from typing import List, Optional

from pydantic import BaseModel, Field, root_validator, validator

import guardrails as gd
from guardrails.utils.pydantic_utils import add_validator
from guardrails.validators import EventDetail, Validator, register_validator


@register_validator(name="is-valid-director", data_type="string")
class IsValidDirector(Validator):
    def validate(self, key, value, schema) -> dict:
        valid_names = [
            "Christopher Nolan",
            "Steven Spielberg",
            "Martin Scorsese",
            "Quentin Tarantino",
            "James Cameron",
        ]
        if value not in valid_names:
            raise EventDetail(
                key,
                value,
                schema,
                f"Value {value} is not a valid director name. "
                f"Valid choices are {valid_names}.",
                None,
            )
        return schema

def validate_naming_thing(name):
    if name == "Christopher Nolan":
        raise ValueError("You are not allowed to choose Christopher Nolan")
    return name


class Director(BaseModel):
    name: str
    # movies: List[Movie]
    movies: List[str]

    # Add guardrails validators
    # _name_validator = add_validator("name", fn=IsValidDirector())
    _name_validator_2 = validator("name", allow_reuse=True)(IsValidDirector(on_fail="exception"))
    _name_validator_3 = validator("name", allow_reuse=True)(validate_naming_thing)

    @validator("name")
    def validate_name(cls, name):
        if name == "Christopher Nolan":
            raise ValueError("You are not allowed to choose Christopher Nolan")
        return name

# guard = gd.Guard.from_pydantic(
#     output_class=Director,
#     prompt=(
#         "Provide detailed information about the top 5 grossing movies from"
#         " {{director}} including release date, duration, budget, whether "
#         "it's a sequel, website, and contact email.\n@complete_json_suffix_v2"
#     ),
#     instructions="You are a helpful assistant only capable of communicating"
#     " with valid JSON, and no other text.\n\n@json_suffix_prompt_examples",
# )

### Getting the class of a registered validator

In [195]:
Director.__fields__["name"].post_validators[0].__name__

'lambda12'

In [30]:
Director.__fields__["name"].post_validators[0].__wrapped__(key="something", value="Christopher Nolan", schema=None)
# Director.__fields__["name"].post_validators[0].__wrapped__(key="something", value="Greta Gerwig", schema=None)

In [151]:
issubclass(type(Director.__fields__["name"].post_validators[0].__wrapped__), Validator)

True

### Getting the function of a registered validator

In [159]:
# Director.__fields__["name"].post_validators[1].__wrapped__("Christopher Nolan")
Director.__fields__["name"].post_validators[1].__wrapped__("Greta Gerwig")

'Greta Gerwig'

In [152]:
issubclass(type(Director.__fields__["name"].post_validators[1].__wrapped__), Validator)

False

In [182]:
Director.__fields__["name"].post_validators[1].__name__

'validate_naming_thing'

In [176]:
hasattr(Director, Director.__fields__["name"].post_validators[1].__name__)

False

### Getting a validator registered with a decorator

In [172]:
Director.__fields__["name"].post_validators[2].__wrapped__

<function __main__.Director.validate_name(cls, name)>

In [174]:
Director.__fields__["name"].post_validators[2].__name__

'validate_name'

In [177]:
hasattr(Director, Director.__fields__["name"].post_validators[2].__name__)

True

In [32]:
from functools import partial


partial_fn = partial(Director.__fields__["name"].post_validators[2].__wrapped__, Director)

In [34]:
partial_fn("Greta Gerwig")
# partial_fn("Christopher Nolan")

'Greta Gerwig'

In [36]:
def validate_naming_thing(name):
    if name == "Christopher Nolan":
        raise ValueError("You are not allowed to choose Christopher Nolan")
    return name


class SomeValidator:
    def __init__(self, acceptable_list: List[str]):
        self.acceptable_list = acceptable_list

    def validate(self, key, value, schema) -> dict:
        if value not in self.acceptable_list:
            raise EventDetail(
                key,
                value,
                schema,
                f"Value {value} is not a valid director name. "
                f"Valid choices are {self.acceptable_list}.",
                None,
            )
        return schema

    def __call__(self, v):
        # if name not in self.acceptable_list:
        #     raise ValueError(f"Name {name} is not in acceptable list {self.acceptable_list}")
        # return name
        return self.validate("dummy_key", v, {"dummy_key": v})["dummy_key"]


class Director(BaseModel):
    name: str
    movies: List[str]

    # Add guardrails validators
    _name_validator = add_validator("name", fn=SomeValidator(acceptable_list=["Christopher Nolan", "Greta Gerwig"]))

In [38]:
Director(name="Martin Sc", movies=["Inception", "Interstellar"])

EventDetail: ('dummy_key', 'Martin Sc', {'dummy_key': 'Martin Sc'}, "Value Martin Sc is not a valid director name. Valid choices are ['Christopher Nolan', 'Greta Gerwig'].", None)

In [40]:
from lxml import etree

In [41]:
etree.Element == etree._Element

False

In [45]:
etree.Element("name")

<Element name at 0x11f051180>

In [43]:
etree._Element

lxml.etree._Element

In [46]:
from lxml.builder import E

In [47]:
el = E("name")

In [49]:
el.set("name", ["Christopher Nolan", "Greta Gerwig"])

TypeError: Argument must be bytes or unicode, got 'list'

In [50]:
import xml.etree.ElementTree as ET

root = ET.Element("root")
child = ET.SubElement(root, "child")
child.set("number", str(42))

In [51]:
root.set("name", ["Christopher Nolan", "Greta Gerwig"])

In [57]:
root.set("other_name", LowerCase(["Christopher Nolan", "Greta Gerwig"]))

In [58]:
root.attrib["other_name"]

In [59]:
import xml.etree.ElementTree as ET

In [60]:
type(root)

xml.etree.ElementTree.Element

In [61]:
ET.fromstring(pretty_print=True)

TypeError: XML() got an unexpected keyword argument 'pretty_print'